### Import libraries

In [1]:
import pandas as pd
import seaborn as sns
import inspect

### Load data

In [2]:
output_dir = 'graphs/constant/'
raw_df = pd.read_csv(f'{output_dir}raw.csv')

### Process results

In [6]:
plot_df = pd.DataFrame()

for nn_method in ['dptt', 'ldp']:
    
    nn_method_df = pd.DataFrame()
    
    # experiment specification
    nn_method_df['early_stopping_level'] = raw_df['early_stopping_level'] - 1
    nn_method_df['eps'] = raw_df['eps']
    nn_method_df['nn_method'] = nn_method
    
    # metrics
    nn_method_df['raw_acc'] = raw_df[f'raw_acc_{nn_method}']
    nn_method_df['top_5_acc'] = raw_df[f'top_5_acc_{nn_method}']
    nn_method_df['precision'] = raw_df[f'precision_{nn_method}']
    nn_method_df['recall'] = raw_df[f'recall_{nn_method}']

    plot_df = pd.concat([plot_df, nn_method_df], ignore_index=True)

### Graph

In [9]:
for metric in ['raw_acc', 'top_5_acc', 'precision', 'recall']: # precision and recall considers top K values relevant

    # plot for both DP-TT and LDP
    plot = sns.lineplot(
        data=plot_df, x='eps', y=f'{metric}', 
        hue='early_stopping_level', style='nn_method', markers=True, dashes=['', (1, 1)],
        palette='deep', alpha=0.75,
    )

    # graph params
    plot.set(
        ylim=(-0.1, 1.1),
        title=f'{metric} vs. total epsilon'
    )

    # move legend
    sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))

    # save plot
    plot.get_figure().savefig(f'{output_dir}{metric}.png', bbox_inches='tight', dpi=400)
    plot.get_figure().clf()

<Figure size 640x480 with 0 Axes>